<a href="https://colab.research.google.com/github/Mandyel/Jogo_Apocal-ptico-_Alura/blob/main/Jogo_Apocaliptico_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-genai
!pip install google-generativeai
!pip install --upgrade google-generativeai

In [ ]:
import random
import time
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

class Jogador:
    def __init__(self, nome):
        self.nome = nome
        self.vida = 100
        self.fome = 50
        self.sede = 50
        self.dias = 3
        self.inventario = ["faca", "cantil de água", "lata de feijão"]
        self.npc_companheiro = None
        self.local_atual = "casa"
        self.encontrou_npc = False
        self.npcs_encontrados = []

def narrar(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Erro ao conectar com o Gemini: {e}")
        return "..."

def mostrar_status(jogador):
    print(f"\n----❤️STATUS❤️----")
    print(f"Nome: {jogador.nome}")
    print(f"Vida: {jogador.vida}/100")
    print(f"Fome: {jogador.fome}/100")
    print(f"Sede: {jogador.sede}/100")
    print(f"Dias restantes: {jogador.dias}")
    print(f"Local atual: {jogador.local_atual.capitalize()}")
    if jogador.npc_companheiro:
        print(f"Companheiro: {jogador.npc_companheiro}")
    print("--------------\n")

def mostrar_inventario(jogador):
    print("\n---💼INVENTÁRIO💼---")
    for i, item in enumerate(jogador.inventario, 1):
        print(f"{i}. {item.capitalize()}")
    print("------------------")

def consumir_item(jogador):
    mostrar_inventario(jogador)
    try:
        escolha = int(input("\nEscolha o número do item que deseja consumir (Escolha 0 para cancelar): "))
        if escolha == 0:
            return

        item = jogador.inventario[escolha-1]

        if "água" in item or "cantil" in item or "refrigerante" in item:
            jogador.sede = min(100, jogador.sede + 30)
            print(f"\nVocê bebeu {item}. Sede aliviada!")
        elif "feijão" in item or "ervilha" in item or "sardinha" in item:
            jogador.fome = min(100, jogador.fome + 30)
            print(f"\nVocê comeu {item}. Fome saciada!")
        elif "kit médico" in item:
            jogador.vida = min(100, jogador.vida + 40)
            print("\nVocê usou o kit médico. Vida recuperada!")
        else:
            print("\nEste item não pode ser consumido.")
            return

        jogador.inventario.pop(escolha-1)
        time.sleep(1)
        mostrar_status(jogador)

    except (ValueError, IndexError):
        print("\nOpção inválida!")

def descartar_item(jogador):
    mostrar_inventario(jogador)
    try:
        escolha = int(input("\nEscolha o número do item que deseja descartar (Escolha 0 para cancelar): "))
        if escolha == 0:
            return

        item = jogador.inventario.pop(escolha-1)
        print(f"\nVocê descartou: {item}")
        time.sleep(1)

    except (ValueError, IndexError):
        print("\nOpção inválida!")

def gerenciar_inventario(jogador):
    while True:
        print("\n--- GERENCIAR INVENTÁRIO ---")
        print("1. Consumir item")
        print("2. Descartar item")
        print("3. Voltar")

        escolha = input("\nEscolha uma opção: ")

        if escolha == "1":
            consumir_item(jogador)
        elif escolha == "2":
            descartar_item(jogador)
        elif escolha == "3":
            break
        else:
            print("\nOpção inválida!")

def ataque_zumbi(jogador):
    dano = random.randint(10, 30)
    chance_escapar = random.randint(1, 100)

    print("\nUm zumbi te ataca!")
    time.sleep(1)

    if jogador.npc_companheiro == "Carlos":
        print("\nCarlos ajuda você no combate!")
        dano = max(5, dano - 15)
        chance_escapar += 20

    print("\n1. Tentar fugir")
    print("2. Lutar com faca")

    escolha = input("\nO que você faz? ")

    if escolha == "1":
        if chance_escapar > 40:
            print("\nVocê conseguiu escapar do zumbi!")
            return True
        else:
            print("\nVocê não conseguiu escapar!")
            jogador.vida -= dano
            print(f"O zumbi te atingiu! Você perdeu {dano} de vida. Vida atual: {jogador.vida}") #linha adicionada
            return False
    elif escolha == "2":
        if "faca" in jogador.inventario:
            chance_vitoria = random.randint(1, 100)
            if chance_vitoria > 30:
                print("\nVocê matou o zumbi com sua faca!")
                return True
            else:
                jogador.vida -= dano
                print(f"\nVocê errou o golpe! O zumbi te atingiu! Você perdeu {dano} de vida.")
                return False
        else:
            print("\nVocê não tem uma faca para lutar!")
            jogador.vida -= dano
            print(f"O zumbi te atingiu! Você perdeu {dano} de vida.")
            return False
    else:
        print("\nVocê ficou paralisado de medo!")
        jogador.vida -= dano
        print(f"O zumbi te atingiu! Você perdeu {dano} de vida.")
        return False

def explorar_local(jogador):
    locais = ["supermercado", "hospital", "lanchonete"]
    itens_disponiveis = {
        "supermercado": ["água", "refrigerante", "lata de feijão", "lata de ervilha", "sardinha"],
        "hospital": ["kit médico", "água", "kit médico"],
        "lanchonete": ["refrigerante", "sardinha", "lata de feijão"]
    }
    npcs = {
        "Carlos": "supermercado",
        "Aline": "hospital",
        "Lucas": "lanchonete"
    }

    print("\nPara onde você quer ir?")
    for i, local in enumerate(locais, 1):
        print(f"{i}. {local.capitalize()}")
    print("4. Voltar para casa")

    try:
        escolha = int(input("\nEscolha um local: "))
        if escolha == 4:
            jogador.local_atual = "casa"
            print("\nVocê volta para casa.")
            return True

        local_escolhido = locais[escolha-1]
        jogador.local_atual = local_escolhido

        if random.randint(1, 100) > 40:
            if not ataque_zumbi(jogador):
                return False

        print(f"\nVocê chegou no {local_escolhido}.")

        if not jogador.encontrou_npc:
            npc_encontrado = None
            for nome_npc, local_npc in npcs.items():
                if local_npc == local_escolhido and nome_npc not in jogador.npcs_encontrados:
                    npc_encontrado = nome_npc
                    break

            if npc_encontrado:
                print(f"\nVocê encontrou {npc_encontrado} no(a) {local_escolhido}!")
                print(f"{npc_encontrado} pode te ajudar na sua jornada.")

                resposta = input(f"\nDeseja levar {npc_encontrado} como companheiro? (sim/não): ").lower()
                if resposta == "s":
                    jogador.npc_companheiro = npc_encontrado
                    jogador.encontrou_npc = True
                    jogador.npcs_encontrados.append(npc_encontrado)
                    print(f"\n{npc_encontrado} agora está com você!")
                    if npc_encontrado == "Aline":
                        jogador.vida = min(100, jogador.vida + 20)
                        print("Aline cura algumas de suas feridas! +20 de vida")
                    elif npc_encontrado == "Carlos":
                        print("Carlos encontra uma arma melhor para você!")
                        if "faca" in jogador.inventario:
                            jogador.inventario.remove("faca")
                        jogador.inventario.append("faca afiada")
                    elif npc_encontrado == "Lucas":
                        jogador.fome = min(100, jogador.fome + 40)
                        print("Lucas te dá algo para comer! +40 de fome")
                else:
                    print(f"\nVocê deixa {npc_encontrado} para trás.")
                    jogador.npcs_encontrados.append(npc_encontrado)

        itens_local = itens_disponiveis[local_escolhido]
        item_encontrado = random.choice(itens_local)

        print(f"\nVocê encontrou: {item_encontrado}!")

        if len(jogador.inventario) < 8:
            jogador.inventario.append(item_encontrado)
            print(f"Você pegou o {item_encontrado}.")
        else:
            print("\nSeu inventário está cheio!")
            mostrar_inventario(jogador)
            resposta = input("Deseja descartar um item para pegar este? (sim/não): ").lower()
            if resposta == "s":
                descartar_item(jogador)
                jogador.inventario.append(item_encontrado)
                print(f"Você pegou o {item_encontrado}.")

        return True

    except (ValueError, IndexError):
        print("\nOpção inválida!")
        return True

def passar_dia(jogador):
    jogador.dias -= 1
    jogador.fome -= 25
    jogador.sede -= 25

    if jogador.npc_companheiro == "Lucas":
        jogador.fome = max(0, jogador.fome + 10)
    if jogador.npc_companheiro == "Aline":
        jogador.vida = min(100, jogador.vida + 5)

    print("\n--- O dia acabou. Você descansa... ---")
    time.sleep(1)

    if random.randint(1, 100) > 70:
        print("\nZumbis atacam sua casa durante a noite!")
        if not ataque_zumbi(jogador):
            return False

    mostrar_status(jogador)

    if jogador.fome <= 0:
        print(narrar(f"Descreva de forma assustadora a morte do personagem {jogador.nome} por fome extrema."))
        return False
    if jogador.sede <= 0:
        print(narrar(f"Descreva de forma assustadora \mas não muito extensa a morte do personagem {jogador.nome} por desidratação."))
        return False
    if jogador.vida <= 0:
        print(narrar(f"Descreva de forma assustadora mas não muito extensa a morte do personagem {jogador.nome} pelos zumbis. Vida atual: {jogador.vida}")) #linha adicionada
        return False

    return True

def jogar():
    while True:
        intro = """
        O mundo como você conhecia acabou.
        Um vírus desconhecido transformou a maior parte da população em zumbis.
        Você é um dos poucos sobreviventes.
        Há rumores de que um helicóptero de resgate chegará em 3 dias.
        Você precisa sobreviver até lá...
        """
        print(narrar(intro))

        nome = input("\n Qual é o seu nome, sobrevivente? ")
        jogador = Jogador(nome)

        print(narrar(f"Descreva de forma assustadora e simples,não muito extensa a cena inicial onde {jogador.nome} está em uma casa com barricadas prestes a serem destruídas, com os zumbis do lado de fora."))

        jogadas_no_dia = 0
        limite_de_jogadas = 5

        while jogador.dias > 0 and jogador.vida > 0:
            mostrar_status(jogador)

            print("\n--- O QUE VOCÊ QUER FAZER? ---")
            print("1. Explorar um local")
            print("2. Gerenciar inventário")
            print("3. Descansar (passar o dia)")

            escolha = input("\nEscolha uma ação: ")
            jogadas_no_dia += 1

            if escolha == "1":
                if not explorar_local(jogador):
                    break
            elif escolha == "2":
                gerenciar_inventario(jogador)
                jogadas_no_dia -= 1
            elif escolha == "3":
                if not passar_dia(jogador):
                    break
                jogadas_no_dia = 0
            else:
                print("\nOpção inválida!")
                jogadas_no_dia -= 1

            if jogadas_no_dia >= limite_de_jogadas and jogador.dias > 0 and jogador.vida > 0:
                print("\nO dia passou e agora será outro dia para sobreviver!")
                if not passar_dia(jogador):
                    break
                jogadas_no_dia = 0

            if jogador.fome <= 0:
                print(narrar(f"Descreva de forma assustadora a morte do personagem {jogador.nome} por fome extrema."))
                break
            if jogador.sede <= 0:
                print(narrar(f"Descreva de forma assustadora a morte do personagem {jogador.nome} por desidratação."))
                break

        if jogador.vida > 0 and jogador.dias <= 0:
            final = f"""
            Depois de 3 longos dias, você ouve o som de um helicóptero se aproximando.
            Você conseguiu sobreviver até o resgate!
            {jogador.nome}, você é um verdadeiro sobrevivente.
            """
            print(narrar(final))
        elif jogador.vida <= 0:
            print("\nVocê não conseguiu sobreviver até o resgate...")
        else:
            print("\nFim do jogo.")

        jogar_novamente = input("\nDeseja jogar novamente? (sim/não): ").lower()
        if jogar_novamente != "sim":
            break

    print("\nObrigado por jogar!")

jogar()
